# TensorBoard: Embedding Visualization

이번 튜토리얼에서는 MNIST 데이터셋(784x1 짜리 벡터)을 embedding visualization시켜 보도록 하겠습니다.

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow as tf

### Load MNIST dataset
MNIST 데이터셋을 불러옵니다. 

In [4]:
mnist = input_data.read_data_sets(train_dir='mnist', reshape=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


### Define Embedding Variable
Embedding 시킬 Variable을 정해줍니다.

In [3]:
embedding_var = tf.Variable(mnist.test.images, name='embedding')

In [4]:
# make log directory if not exists
log_dir = 'logs/'
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

# gpu configuration 
gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=gpu_config) 

In [5]:
tf.global_variables_initializer().run()
saver = tf.train.Saver()
saver.save(sess, save_path=log_dir+'model.ckpt', global_step=None)

# (required) add embedding data
summary_writer = tf.summary.FileWriter(logdir=log_dir, graph=tf.get_default_graph())
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

In [7]:
# (optional) add meta data such as labels and sprite image 
metadata_path = 'logs/metadata.tsv'
with open(metadata_path, 'w') as f:
    f.write('Label\t Label2\n')
    for i, label in enumerate(mnist.test.labels):
        f.write('{}\t{}\n'.format(label, label))
embedding.metadata_path = metadata_path

In [8]:
# visualize embedding projector
projector.visualize_embeddings(summary_writer, config)